In [10]:
## Impoerting necessary Libraries
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import os

#import chrome webdriver
browser = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')

browser.get('https://terminus.isprimefx.com/login')
time.sleep(1)
# Enter Login Creds for Terminus
elementID = browser.find_element_by_id('username')
elementID.send_keys('operations@alphatrade.com.au')

elementID = browser.find_element_by_xpath('//*[@id="loginForm"]/div[2]/input')
elementID.send_keys('promptshadeimpose')

elementID.submit()
browser.fullscreen_window()
time.sleep(1)
dload = browser.find_element_by_id('dashboardPositionsDownload')
# time.sleep(4)
dload.click()
#Open login page Arms
browser.get('https://arms3.onezero.com/login')
time.sleep(1)

# Enter Login Creds for Arms
elementID = browser.find_element_by_id('username')
elementID.send_keys('operations@alphatrade.com.au')

elementID = browser.find_element_by_id('password')
elementID.send_keys('Password1!')
# time.sleep(1)
elementID.submit()
# browser.fullscreen_window()
time.sleep(1)
dropdown = browser.find_element_by_xpath('//*[@id="topNav"]/div/div/div/ul/li[2]/a')
dropdown.click()
li = dropdown.parent.find_elements_by_tag_name('li')
li[5].click()
checkbox = browser.find_element_by_id('form_broker_database_connections_0')
checkbox.click()
time.sleep(3)
select = Select(browser.find_element_by_id('form_group_by'))
select.select_by_value('maker')
time.sleep(7)
browser.fullscreen_window()
time.sleep(5)
exportCsv = browser.find_element_by_css_selector('#exposureBox > div > div.pull-right > button')
time.sleep(2)
exportCsv.click()

#Open login page GCEX Web Traders
browser.get('https://live.gc.exchange/')
time.sleep(5)

# Enter Login Creds for Gcex
elementID = browser.find_element_by_xpath('/html/body/div[3]/div[3]/div/div[1]/div[2]/form/div[1]/input')
elementID.send_keys('Alpha_Ops_3')

elementID = browser.find_element_by_xpath('/html/body/div[3]/div[3]/div/div[1]/div[2]/form/div[2]/input')
elementID.send_keys('4LUn3a5ub#PW')
button = browser.find_element_by_xpath('/html/body/div[3]/div[3]/div/div[1]/div[2]/form/div[3]/button[1]')
button.click()
time.sleep(2)
browser.fullscreen_window()
time.sleep(9)
pnl = browser.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[1]/div/ul/li[1]/div/button[3]')
pnl.click()
time.sleep(10)
browser.find_element_by_xpath('/html/body/div[3]/div[4]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[1]/div/div/button[3]').click()
time.sleep(10)
browser.close()

### Position Matching Analysis Starts here
## Load the data
list_for_ter = []
lis_for_gcx = []
lis_for_arms = []
terminus = os.listdir(r'C:\Users\oazed\Downloads')

for fname in terminus:
    if fname.startswith('_2022'):
        list_for_ter.append(fname)

for fname in terminus:
    if fname.startswith('DataGrid'):
        lis_for_gcx.append(fname)
        
for fname in terminus:
    if fname.startswith('Hub Positions'):
        lis_for_arms.append(fname)
        
temp = list_for_ter[-1]
temp_gcx = lis_for_gcx[-1]
temp_arms = lis_for_arms[-1]
term = pd.read_csv('C:\\Users\\oazed\\Downloads\\'+temp)
gcx = pd.read_excel('C:\\Users\\oazed\\Downloads\\'+temp_gcx)
arms = pd.read_csv('C:\\Users\\oazed\\Downloads\\'+temp_arms)


arms = arms[arms['Grouping'] == 'All Instruments']
arms = arms[['Core Symbol', 'Total Position']]
gcx = gcx[['Security', 'NetPosition']]
term = term[['Currency Pair', 'Base Amount']]
term = term.rename(columns={'Currency Pair': 'Security', 'Base Amount':'NetPosition'})
all_lps = pd.concat([gcx,term])

## Changing to numeric
all_lps['NetPosition'].replace(to_replace=",", value="", regex=True, inplace=True)
all_lps['NetPosition'] = all_lps['NetPosition'].apply(pd.to_numeric)
arms['Total Position'].replace(to_replace=",", value="", regex=True, inplace=True)
arms['Total Position'] = arms['Total Position'].apply(pd.to_numeric)

all_lps = all_lps.groupby(["Security"]).agg({"NetPosition":"sum"}).reset_index()

all_lps = all_lps[all_lps['NetPosition']!=0].reset_index()
arms = arms[arms['Total Position']!=0].reset_index()
arms = arms.drop(['index'],axis=1)

### Changing the Index Names to match LPS
eods_dict= {
    'US500' : 'SPM/USD',
    'NAS100': 'NDM/USD',
    'US30' : 'DJI/USD',
    'AUS200': 'AXI/AUD',
    'EUSTX50':'STX/EUR',
    'HK50': 'HSI/HKD',
    'JPN225': 'JPU/JPY',
    'UK100' : 'UKI/GBP',
    'GER30': 'GEM/EUR',
    'CN50' : 'CNI/CNY',
    'FRA40':'FRI/EUR'
}

def abar_banao(x):
    ins = x['Core Symbol']
    if ins in eods_dict.keys():
        return eods_dict[ins]
    else:
        return ins
arms['Core Symbol'] = arms.apply(abar_banao,axis=1)

merged = pd.merge(arms, all_lps, left_on = 'Core Symbol', right_on = 'Security', how = 'outer')
merged['Difference'] = merged['Total Position'] - merged['NetPosition']
merged.to_csv('C:\\Users\\oazed\\OneDrive\\Desktop\\Position\\position.csv')
os.remove("C:\\Users\\oazed\\Downloads\\"+temp)
os.remove("C:\\Users\\oazed\\Downloads\\"+temp_gcx)
os.remove("C:\\Users\\oazed\\Downloads\\"+temp_arms)
